<a href="https://colab.research.google.com/github/Fr33zz/mw/blob/master/18%20Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from datetime import date
import datetime

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
orders = gc.open_by_url('https://docs.google.com/spreadsheets/d/106Mk7_n9iXmCB-fuPFgpWsV1t7JtoAdPaePi5x8BFTM')

payments = gc.open_by_url('https://docs.google.com/spreadsheets/d/1OKkFUxN-a55K52Z6QO0wnZg9SB-_6d3VZ_BXDCYsGYc')

closed = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Gs7OkvQLNpt5IDTrsP5PBWbrSTO89zOPysbpbp7uzrQ')

sch_pay = gc.open_by_url('https://docs.google.com/spreadsheets/d/1AkH2q9AGbFuaXW3fUlrvvzcObFAr0vOkYdr2sNb4uLw')

opened = gc.open_by_url('https://docs.google.com/spreadsheets/d/1xUc4ayCcVNpkT1Bcx8_Hk84g02iM3U5GkxSSbZMMGAk')

charge = gc.open_by_url('https://docs.google.com/spreadsheets/d/1gOgXDjkxdjNaE-O-jprV77Tucdqm8uE-vg0NHDxP8xk')

partners = gc.open_by_url('https://docs.google.com/spreadsheets/d/1iLbCMeMZ19wpOyVZCVUUvXKVdRYLveNwrsr6FoJ5FJQ')

def get_data(spreadsheet, list_number=0):
  ind = pd.DataFrame(spreadsheet.worksheets()[list_number].get_all_values())
  ind = ind.rename(columns=ind.iloc[0]).drop(index=0).reset_index().drop(columns='index')
  return ind

def get_titles(x, type_=1):
  '''type = {0:"list", 1:"df"}'''
  if type_==0:
    return [i.title for i in x.worksheets()]
  if type_==1:
    return pd.DataFrame([i.title for i in x.worksheets()])


def to_num(data, columns, print_=False, list_=False):
  d = data.copy()
  l = []
  for i in columns:
    try:       
      if len(d[i].shape)==1:
        d[i] = d[i].map(lambda x: str(x) if x!='' and x==x else np.nan)
        d[i] = d[i].map(lambda x: '.'.join(x.split(','   )) if x!='' and x==x else np.nan)
        d[i] = d[i].map(lambda x:  ''.join(x.split(' '   )) if x!='' and x==x else np.nan)
        d[i] = d[i].map(lambda x:  ''.join(x.split('\xa0')) if x!='' and x==x else np.nan)
      if len(d[i].shape)==2:
        d[i] = d[i].applymap(lambda x: str(x) if x!='' and x==x else np.nan)
        d[i] = d[i].applymap(lambda x: '.'.join(x.split(','   )) if x!='' and x==x else np.nan)
        d[i] = d[i].applymap(lambda x:  ''.join(x.split(' '   )) if x!='' and x==x else np.nan)
        d[i] = d[i].applymap(lambda x:  ''.join(x.split('\xa0')) if x!='' and x==x else np.nan)
      
      d[i] = d[i].astype('float')
    except:
      l.append(i)
      if print_:
        print(print_,' : ', i)
      else:
        None
  if list_:
    return l, d
  else:
    return d


def to_dates(data, columns, print_=False, list_=False):
  d = data.copy()
  l = []
  for i in columns:
    try:
      
      d[i] = d[i].replace({'':np.nan})
      d[i] = d[i].str.split('T').map(lambda x: x[0] if x==x else x)
      #d[i] = d[i].str.split('-').map(lambda x: int(x[2]) if x==x else x)
      #d[i] = pd.to_datetime(d[i], format='%Y-%m-&d')
      d[i] = pd.to_datetime(d[i])
    
    except:
      l.append(i)
      if print_:
        print(print_, ' : ', i)
  if list_:
    return l, d
  else:
    return d

#for i in dates:
#  d[i] = pd.to_datetime(d[i].replace({'':np.nan}), format='%d.%m.%Y')

In [ ]:
'''
--BDATES

select
    ca.masked_credit_agreement_id
    , lp.birth_date
    , cg.closed_at
from credit_applications ca
left join local_passports lp on lp.id = ca.local_passport_id
left join credit_agreements cg on cg.credit_application_id = ca.id
where ca.state = 'approved'
'''

"\nselect\n    ca.masked_credit_agreement_id\n    , lp.birth_date\n    , cg.closed_at\nfrom credit_applications ca\nleft join local_passports lp on lp.id = ca.local_passport_id\nleft join credit_agreements cg on cg.credit_application_id = ca.id\nwhere ca.state = 'approved'\n"

In [ ]:
op = get_data(gc.open_by_url('https://docs.google.com/spreadsheets/d/1Hw3w42gDRGaAGtQBDfflO8Vq0IZbjRYXRJUp3PgMV9U/'))
bd = pd.read_csv('/content/drive/My Drive/analytics/18 cb report 2020.11.16/bdates.csv')

ord = get_data(orders, 0)
ord = ord[ord['Тип заказа']=='заем']

ord = pd.merge(left=ord, right=op, left_on='№', right_on='№ договора', how='left')
ord = pd.merge(left=ord, right=get_data(closed)[['Дата фактическая', '№']], on='№', how='left')
ord = pd.merge(left=ord, right=bd, left_on='№', right_on='masked_credit_agreement_id', how='left')

ord = to_num(ord, ord.columns)

ord['birth_date'] = ord['birth_date'].map(lambda x: x.split('T')[0] if x==x else x)
ord['closed_at'] = ord['closed_at'].map(lambda x: '.'.join(x.split('T')[0].split('-')[::-1]) if x==x else x)

ind = ord[ord['Дата фактическая'].isna()].index
ord.loc[ind, 'Дата фактическая'] = ord.loc[ind, 'closed_at']

In [ ]:
ord.tail()
#ord.iloc[:,48:]

,Открытие,Закрытие,Тип заказа,Повторный_x,Партнер_x,№,№ (партнер),Клиент,Пол,Возраст,Паспорт,Загранпаспорт,Индекс,Телефон,Почта,Операционная система,Браузер,Провайдер первого платежа,"Сумма заказа, руб.","Сумма заказа партнера, руб.","Сумма долга, руб.","Сумма платежа, руб.","Переплата, руб.","Переплата, %","ПСК, %","Срок, дней",Первый платеж,"Сумма, руб.","Тело платежа, руб.","Комиссия, руб.","Скидка, руб.","Эквайринг, руб.",Второй платеж,"Сумма, руб.","Тело платежа, руб.","Процент, руб.","Комиссия, руб.","Скидка, руб.","Эквайринг, руб.",Третий платеж,...,Дата окончания договора,Срок кредита / займа,Валюта договора,Процентная ставка,Полученная сумма заемщиком,Ежемесячный взнос,Сумма начисленных процентов,Дата начала просрочки,Срок просрочки (в днях),Категория просрочки,Сумма оплат основного долга (всего),Сумма оплат процентов (всего),Сумма оплат штрафов (всего),Дата последнего платежа,Сумма последнего платежа,Сумма оплат основного долга,Сумма оплат процентов,Сумма оплат штрафов / пени,Задолженность по основному долгу,Задолженность по процентам,Задолженность по штрафам,Задолженность по комиссии,Общая сумма задолженности,Текущая задолженность по основному долгу,Текущая задолженность по процентам,Текущая задолженность по комиссии,"Резерв по основному долгу, %",Резерв по основному долгу,"Резерв по процентам, %",Резерв по процентам,Просроченная задолженность по телу,Просроченная задолженность по процентам,Просроченная задолженность по штрафам,Просроченная задолженность по комиссии,Госпошлина,,Дата фактическая,masked_credit_agreement_id,birth_date,closed_at
15168,17.11.2020,16.01.2021,заем,Нет,UFS.Travel,MW333395036,110790108,СтепановаИ.И.,Ж,31.0,4.008722e+09,неуказан,неуказан,7.981798e+10,Sapronova89@mail.ru,Android10,ChromeMobile81.0.4044.138,MAP,4134.0,3418.6,2040.6,1378.0,715.4,20.93,127.0,60.0,17.11.2020,1378.0,1378.0,0.0,0.0,22.74,17.12.2020,1378.0,969.69,213.00,195.31,0.0,22.74,16.01.2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MW333395036,1989-02-13,NaN
15169,17.11.2020,16.01.2021,заем,Да,Азимут,MW333395038,1kPpIUtwfzCbSCOum5cfQ0OFlXR,ОвчаренкоС.Р.,М,34.0,6.018472e+09,неуказан,346421,7.905424e+10,ovcharenko096@gmail.com,iOS14.0,ChromeMobileiOS83.0.4103.88,MAP,4149.0,3430.0,2047.0,1383.0,719.0,20.96,127.0,60.0,17.11.2020,1383.0,1383.0,0.0,0.0,22.82,17.12.2020,1383.0,972.73,213.67,196.60,0.0,22.82,16.01.2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MW333395038,1986-10-16,NaN
15170,17.11.2020,16.01.2021,заем,Да,Азимут,MW333395050,1kQ7uyg44UqiUcJEx1Rlq8TyunX,ФароновА.В.,М,22.0,8.518557e+09,неуказан,358001,7.961540e+10,afaronov11@gmail.com,iOS12.4.8,MobileSafari12.1.2,MAP,6933.0,5730.0,3419.0,2311.0,1203.0,20.99,127.0,60.0,17.11.2020,2311.0,2311.0,0.0,0.0,38.13,17.12.2020,2311.0,1624.70,356.89,329.41,0.0,38.13,16.01.2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MW333395050,1998-09-02,NaN
15171,17.11.2020,16.01.2021,заем,Да,Азимут,MW333395078,1kQVIHojYbzFaQ4TUp20ZrQkSLu,ЗахаровС.К.,М,25.0,5.815205e+09,неуказан,180014,7.999976e+10,frost-1231@mail.ru,Android10,ChromeMobile86.0.4240.185,MAP,4305.0,3560.0,2125.0,1435.0,745.0,20.93,127.0,60.0,17.11.2020,1435.0,1435.0,0.0,0.0,23.68,17.12.2020,1435.0,1009.80,221.81,203.39,0.0,23.68,16.01.2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MW333395078,1995-01-21,NaN
15172,17.11.2020,16.01.2021,заем,Да,OZON.travel,MW333395082,14312268-0106-01,КварацхелияС.И.,Ж,27.0,4.519457e+09,неуказан,142530,7.903581e+10,sabinakvarc717@yandex.ru,iOS13.6,MobileSafari13.1.2,MAP,14967.0,12371.0,7382.0,4989.0,2596.0,20.98,127.0,60.0,17.11.2020,4989.0,4989.0,0.0,0.0,82.32,17.12

In [ ]:
ord['Ф.И.О. / наименование заемщика'] = ord['Клиент'].map(lambda x: ' '.join([x[:-4], x[-4:-2], x[-2:]]))
ord['Дата заключения договора'] = ord['Открытие']
ord['Дата рождения заемщика'] = ord['birth_date']
ord['Номер договора'] = ord['№']
ord['Категория потребительского займа (в соответствии с Указанием Банка России от 01.04.2019 № 5112-У)'] = 'Потребительские микрозаймы с иным обеспечением до 365 дней включительно'
ord['Цель займа'] = 'оплата авиа-, ж/д билетов'
ord['Дата выдачи займа'] = ord['Открытие']
ord['Первоначальная предполагаемая дата возврата займа'] = ord['Закрытие']
ord['Срок действия договора (дни)'] = ord['Срок, дней']
ord['Сумма займа по договору'] = ord['Сумма долга, руб.']
ord['Значение ПСК'] = ord['ПСК, %']
ord['Процентная ставка '] = ord['ПСК, %']
ord['Предложены платные дополнительные услуги (да/нет)'] = 'да'
ord['Наименование платной дполнительной услуги '] = 'Автоплатеж'
ord['Стоимость платной дополнительной услуги'] = ord['Комиссия, руб.'].sum(axis=1)
ord['Оплата дополнительной услуги (собственные средства заемщика/из суммы займа)'] = 'Собственные средства'
ord['Размер комиссионного  вознаграждения Общества по агентскому догововору (в руб.)'] = 0

ord['Сумма основного долга по состоянию на 01.11.2020'] = ord['Задолженность по основному долгу']
ord['Сумма начисленных процентов по состоянию на 01.11.2020'] = ord['Задолженность по процентам']
ord['Сумма штрафов и пени'] = ord['Задолженность по штрафам']
ord['Общая сумма задолженности '] = ord['Общая сумма задолженности']

ord['Фактическая дата погашения займа'] = ord['Дата фактическая']
ord['Исполнен/просрочен'] = ord['Срок просрочки (в днях)'].map(lambda x: 'просрочен' if x!=0 and x==x else 'исполнен')
ord['Сумма денежных средств, перечисленных заемщику (на счет, карты, онлайн кошелек, иное (руб.)'] = 0
ord['Наличие распоряжения  заемщика на передачу третьему лицу суммы по договору займа  (да/нет)'] = 'да'
ord['Наименование и ИНН получателя денежных средств, перечисляемых по распоряжению заемщика '] = ord['Партнер_x']
ord['Сумма денежных средств, перечисленных на расчетный счет третьего лица, по распоряжению заемщика (руб.)'] = ord['Сумма заказа партнера, руб.']
ord['Права по договору займа уступлены (да/нет)'] = 'нет'

In [ ]:
d = ord[ord['Дата заключения договора'].map(lambda x: len(x.split('08.2020'))==2 or len(x.split('09.2020'))==2 or len(x.split('10.2020'))==2)].iloc[:,-28:]

part = {'Aviakassa.com': '"E-commerce Corporation LLP(Citibank, N.A., London Branch)" ИНН 12799',
        'Biletix': 'ООО "Билетикс.РУ" ИНН 7730606598',
        'City.Travel': 'СИТИТРЭВЕЛ ЛТД ИНН 9909409904',
        'Mego.travel': 'ООО МЕГО ТРЕВЕЛ РУС ИНН 7842432220',
        'OZON.travel': 'ООО "Интернет Трэвел" ИНН 7705859041',
        'Tickets.ru': 'ООО ТИКЕТС.РУ (Тикетс Ру Про) ИНН 7802720863',
        'UFS.Travel': 'ООО УФС ИНН 7708510731',
        'Азимут': 'Авиакомпания Азимут АО ИНН 2312218415',
        'СвязнойТрэвел': '"Билетикс.РУ" ИНН 7730606598'}

d['Наименование и ИНН получателя денежных средств, перечисляемых по распоряжению заемщика ']\
   = d['Наименование и ИНН получателя денежных средств, перечисляемых по распоряжению заемщика '].map(lambda x: part[x])


In [ ]:
op = get_data(gc.open_by_url('https://docs.google.com/spreadsheets/d/1BZZU7jpEypLkzvJkSOb0W50iHOeMSKobvFB4EKBPTNo'))[['№ договора', 'Задолженность по основному долгу', 'Задолженность по процентам', 'Задолженность по штрафам', 'Общая сумма задолженности']]

op = to_num(op, op.columns)
d = pd.merge(left = d, right = op, left_on='Номер договора', right_on='№ договора', how='left')

op = op.rename(columns={'Задолженность по основному долгу':'telo',
                        'Задолженность по процентам':'proc',
                        'Задолженность по штрафам':'pen',
                        'Общая сумма задолженности':'sum'})

d = pd.merge(left = d, right = op, left_on='Номер договора', right_on='№ договора', how='left')

d['telo'] -= d['Сумма основного долга по состоянию на 01.11.2020']
d['proc'] -= d['Сумма начисленных процентов по состоянию на 01.11.2020']
d['pen']  -= d['Сумма штрафов и пени']
d['sum']  -= d['Общая сумма задолженности ']

d['proc'] = d['proc'].map(lambda x: x if x==x and x>0 else 0)
d['pen']  = d['pen'].map( lambda x: x if x==x and x>0 else 0)

d['Сумма начисленных процентов по состоянию на 01.11.2020'] -= 2*d['proc']
d['Сумма штрафов и пени'] -= 2*d['pen']
d['Общая сумма задолженности '] -= 2*d['pen'] + 2*d['proc']

In [ ]:
drop2 = ['MW333384774',
 'MW333385072',
 'MW333386694',
 'MW333386962',
 'MW333388629',
 'MW333388891',
 'MW333389311',
 'MW333381808',
 'MW333382062',
 'MW333382073',
 'MW333393221']

drop2 = d[d['Номер договора'].isin(drop2)].index
d = d.drop(index=drop2)

In [ ]:
path = '/content/drive/My Drive/analytics/18 cb report 2020.11.16/result.xlsx'
d.to_excel(path)

In [ ]:
s ='''сделать проверку формата данных для модели
сделать передачу данных в мегафон
сделать прием данных от мегафона
сделать препроцессинг заявки перед мегафоном
сделать препроцессинг данных после мегафона
сделать оценку кол-ва заявок в месяц
оценить качество моделей мегафона
оценить качество модели без мегафона
обучить более точную модель
получить инфу для интеграции с мегафоном
подписать договор с мегафоном
сделать метрики качества для моделей
сделать логику одобрения вне смены
узнать как добыть данные из нбки
узнать как добыть данные из пенсионного фонда
узнать как добыть данные из финкарты
узнать как добыть данные из скористы
решить как будет выкатывать на прод'''


[print('скоринг :: '+i) for i in s.split('\n')]

скоринг :: сделать проверку формата данных для модели
скоринг :: сделать передачу данных в мегафон
скоринг :: сделать прием данных от мегафона
скоринг :: сделать препроцессинг заявки перед мегафоном
скоринг :: сделать препроцессинг данных после мегафона
скоринг :: сделать оценку кол-ва заявок в месяц
скоринг :: оценить качество моделей мегафона
скоринг :: оценить качество модели без мегафона
скоринг :: обучить более точную модель
скоринг :: получить инфу для интеграции с мегафоном
скоринг :: подписать договор с мегафоном
скоринг :: сделать метрики качества для моделей
скоринг :: сделать логику одобрения вне смены
скоринг :: узнать как добыть данные из нбки
скоринг :: узнать как добыть данные из пенсионного фонда
скоринг :: узнать как добыть данные из финкарты
скоринг :: узнать как добыть данные из скористы
скоринг :: решить как будет выкатывать на прод


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]